In [ ]:
from flask import Flask, request, render_template, flash, redirect, url_for
from pymongo import MongoClient

# Initialize the Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Needed for flashing messages

# Initialize the MongoDB client and specify the database and collection
client = MongoClient('mongodb://localhost:27017/')  # Connect to the MongoDB server
db = client['user_data_db']  # Use a database called 'user_data_db'
collection = db['user_data']  # Inside the database, use a collection called 'user_data'

# Route to render the study overview and consent
@app.route('/')
def overview():
    return render_template('consent.html')

# Route to render the actual form after consent
@app.route('/form')
def form():
    return render_template('index.html')

# Data collection route
@app.route('/collect_data', methods=['POST'])
def collect_data():
    try:
        # Get user input
        age = request.form['age']
        gender = request.form['gender']
        total_income = request.form['total_income']
        
        # Gather expenses from the form
        expenses = {
            'utilities': request.form.get('utilities', 0),
            'entertainment': request.form.get('entertainment', 0),
            'school_fees': request.form.get('school_fees', 0),
            'shopping': request.form.get('shopping', 0),
            'healthcare': request.form.get('healthcare', 0)
        }

        # Validate that essential fields are not empty
        if not age or not gender or not total_income:
            flash("Please fill in all required fields: Age, Gender, and Total Income.", "error")
            return redirect('/form')

        # Convert fields to the correct type and validate them
        try:
            age = int(age)
            total_income = float(total_income)
            expenses = {k: float(v) if v else 0 for k, v in expenses.items()}
        except ValueError:
            flash("Please enter valid numeric values for Age, Total Income, and Expenses.", "error")
            return redirect('/form')

        # Create a dictionary of the data
        user_data = {
            'age': age,
            'gender': gender,
            'total_income': total_income,
            'expenses': expenses
        }

        # Insert the data into MongoDB
        collection.insert_one(user_data)

        # Redirect to a success page after data submission
        return redirect(url_for('success'))

    except Exception as e:
        flash(f"An error occurred: {str(e)}", "error")
        return redirect('/form')

# Success route to show the success message and provide a button to return to the form
@app.route('/success')
def success():
    return render_template('success.html')

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
